# Fs-Monty-Hall

Author: Evelina Gabasova  
Transcribed to a Jupyter notebook by Ian Buckley.

## Overview

This notebook, which can be conveniently [hosted on Azure](https://notebooks.azure.com/), is a transcription of the [F#](http://fsharp.org/) program `comp_expr_monty_hall` developed by Evelina Gabasova as part of her presentation [How to look like a statistician: a developer's guide to probabilistic programming](https://channel9.msdn.com/Events/FSharp-Events/fsharpConf-2018/06), part of [FSharpConf](http://fsharpconf.com/) on 6 April 2018. This particular example solves the [**Monty Hall problem**](https://en.wikipedia.org/wiki/Monty_Hall_problem), making use of a common design pattern in functional programming, useful for expressing workflows, called the [**computation expression**](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/computation-expressions) (also known as in math & computer science circles & by the Haskell & Scala communities as the [**monad**](https://en.wikipedia.org/wiki/Monad_(functional_programming)). 

Computational expressions are explained in detail on [F# for fun or profit](https://fsharpforfunandprofit.com/series/computation-expressions.html). Probabilistic programming is an important aspect of machine learning unifying general purpose programming with probabilistic modeling. It is a great application for computation expressions, since it has been known for a while that [the monad design pattern is ideal to describe probabilistic computations](https://www.cs.tufts.edu/~nr/pubs/pmonad.pdf). 

The Monty Hall problem has been [solved in Haskell](https://web.engr.oregonstate.edu/~erwig/papers/PFP_JFP06.pdf) and Scala: [pfp-scala](https://github.com/andresilva/pfp-scala), [odds](http://lampwww.epfl.ch/~sstucki/talks/rsc-meetup13/rsc-meetup13-slides.pdf). 

Frameworks for [probabilistic programming](http://www.probabilistic-programming.org/wiki/Home):
* **Scala** include [Figaro](https://github.com/p2t2/figaro), [odds](https://github.com/sstucki/odds) and [Pfennig](https://github.com/noelwelsh/pfennig)
* **Haskell** are described [here](https://wiki.haskell.org/Probabilistic_Functional_Programming)
* **.net** include [Infer.NET](http://infernet.azurewebsites.net/) & specifically in F#: [Infer.NET Fun](https://www.microsoft.com/en-us/research/project/infer-net-fun/)

https://channel9.msdn.com/Events/FSharp-Events/fsharpConf-2018/06   
https://github.com/evelinag/probabilistic-programming/tree/master/code   

##### Notebook extensions

Because this is a long notebook, & a mixture of valuable code & less valuable experiments, it is a really good idea to turn on a couple of notebook extensions **Edit > nbextensions config** to open a new browser tab, & then select the following extensions:
* Collapsible headings
* Intitialization cells (allows you to conveniently run only specific cells   

Having selected those check boxes, reload the tab containing this notebook for the changes to take effect.

## `Load` & `open` .net packages using `Paket`

### Steps when running on Azure

#### Run once in a given Azure library to create script

In [2]:
#load "Paket.fsx"

When using Azure, run the following to generate the file `Paket.Generated.Refs.fsx`, to be `load`ed each time that the notebook is run with a fresh kernel. The file is put in either 
`/home/nbuser/IfSharp/bin` or 
`/home/nbuser/library`

The following installs the selected packages. It may take a minute to run on first use with a particular kernel.

In [3]:
Paket.Package
  [ "MathNet.Numerics"
    "MathNet.Numerics.FSharp"
    "FSharp.Data"
  ]

#### Run always for restarted kernel 

 `Load` the file `Paket.Generated.Refs.fsx` each time that the notebook is run with a fresh kernel when on Azure.

In [1]:
#load "Paket.Generated.Refs.fsx"

Could not load file '/home/nbuser/IfSharp/bin/.paket/load/main.group.fsx' because it does not exist or is inaccessible

### Steps for local execution

In [5]:
// On Azure, use the (Unix) terminal window to find these files in ~/IfSharp/bin/
#r "packages/MathNet.Numerics/lib/net40/MathNet.Numerics.dll"
#r "packages/FSharp.Data/lib/net45/FSharp.Data.dll"

### Common steps for Azure or local execution

In [2]:
open System
open MathNet.Numerics
open FSharp.Data

The namespace or module 'MathNet' is not defined. Maybe you want one of the following:
   Math

## Define types, computation expressions, functions, models

### Define types & helper functions for outcomes, distributions

#### `Door`

The `Door` type describes the possible outcomes of the experiment (opening a door in the game). I.e. the "sample space".

In [3]:
type Door = Goat | Car

#### `MontyHallValue`

The discrete distribution of the events is modelled as a sequence of records, each containing an outcome value & its probability. E.g. for the three door game: `[Car;Goat;Goat]` all with probability $1/3$.

In [4]:
type MontyHallValue = {
    Value: Door
    Probability: float }

Forkmann calls this `Outcome`: http://www.navision-blog.de/2011/08/15/some-special-monads-in-f-part-3-of-n-distributionmonad/.

#### `Distribution`

A distribution is modelled as a collection (`seq`) of `MontyHallValue`s.

In [5]:
type Distribution = MontyHallValue seq

#### Helper functions for uniform & certain distributions

In [6]:
let uniformDistribution (values: Door seq) = 
    let l = float (Seq.length values)
    values |> Seq.map (fun v -> {Value=v; Probability = 1.0/l})

In [7]:
let certainly value = seq [{Value=value; Probability =1.0}]

### Computation expression for probabilistic computations

#### `ProbabilisticComputation`

A `ProbabilisticComputation` is a (generic) union type consisting of either:
* `Sample` is a tuple of 
    * a `Distribution` (over `MontyHallValue`s) and 
    * a function that maps an input of type `Door` to a `ProbabilisticComputation`
* `Return` is just an alias for the provided wrapped type `'T`

In [8]:
type ProbabilisticComputation<'T> = 
  | Sample of Distribution * (Door -> ProbabilisticComputation<'T>)
  | Return of 'T   

#### `ProbabilisticComputationBuilder`

The computation expression builder must implement the `Return` and `Bind` methods.
* **`Return`** Lifts a single value into the elevated world `a -> E<a>`
* **`Bind`**  Allows you to compose world-crossing (“monadic”) functions `(a->E<b>) -> E<a> -> E<b>`

In [9]:
type ProbabilisticComputationBuilder() = 
  member x.Return(v) = Return v
  member x.Bind(dist, f) = Sample(dist, f) 

Create an instance of the builder class for use below defining strategy models.

In [10]:
let prob = ProbabilisticComputationBuilder()

#### Diagram of `bind` function from F# for fun & profit

Diagram of `bind` function from F# for fun & profit. See [bind](https://fsharpforfunandprofit.com/posts/elevated-world-2/#bind). In the computation expression, after a dose of syntactic sugar, `bind` method appears as `let!`. .
In this concrete example the correspondences are:
* the *elevated world* is a (generic) type `ProbabilisticComputation<Door>` (so either a `Sample` or a `Return`).
* the type `a` is always an outcome of type `Door`
* ditto for type `b`??

![Image of bind function from F# for fun & profit](https://fsharpforfunandprofit.com/assets/img/vgfp_bind.png)

### Monty Hall models

Define `ProbabilisticComputation`s using the `ProbabilisticComputationBuilder` instance `prob` to represent the strategies of staying with the initial door, or switching to the one proposed by Monty. When the contestant stays, `bind` is called once. When she switches, it is called twice.

In [11]:
let montyHallStay = prob {
    let! initialDoor = uniformDistribution [Car;Goat;Goat]
    return initialDoor}

let montyHallSwitch = prob {
    let! initialDoor = uniformDistribution [Car;Goat;Goat]
    let! switchedDoor = 
        match initialDoor with
        | Car -> certainly Goat
        | Goat -> certainly Car
    return switchedDoor}

The two strategies are (initially) of type `ProbabilisticComputation.Sample`, consisting of a tuple containing a distribution (sequence of `Value`-`Probability` record pairs), & a function. 

In [86]:
montyHallStay

Sample
  (seq
     [{Value = Car;
       Probability = 0.3333333333;}; {Value = Goat;
                                      Probability = 0.3333333333;};
      {Value = Goat;
       Probability = 0.3333333333;}],<fun:montyHallStay@3-3>)

### Helper functions to `enumerate` outcomes & `summarize` the probabilities of winning the car (or goat)

#### `enumerate`

Inputs are the values (outcomes) of a distribution & a probabilistic computation.  
The function `enumerate` is recursive, so explores the tree. 

In [62]:
let rec enumerate values comp = seq {
    match comp with 
    | Sample (dist:Distribution,f) ->
        printfn "============ ProbabilisticComputation is a Sample"
        // Loop over the MontyHallValues in the Distribution
        for option in dist do 
            yield! enumerate (option::values)(f option.Value)
    | Return (v:Door) ->
        // Print what's happening
        values |> List.rev
        |> List.iteri (fun i mh -> printfn "PC is a Return %d: %A(%f)" (i+1) mh.Value mh.Probability)
        // Yield a tuple with the outcome (Car/Goat) & its probability
        yield v,
            (1.0, values)
            ||> List.fold (fun acc x -> acc * x.Probability)}

Has type:

`val enumerate :
  values:MontyHallValue list ->
    comp:ProbabilisticComputation<'a> -> seq<'a * float>`

#### `summarize`

Use `groupBy` to aggregate over outcomes of the same value (e.g. Goat or Car). Output is a list of tuples: e.g. `[(Car, 0.333); (Goat, 0.667)]`

In [51]:
let summarize dist = 
    dist
    |> Seq.groupBy fst   // Want two bins: for Car & Goat
    |> Seq.map (fun (value, xs:('a * float) seq) ->
        value, xs |> Seq.sumBy snd)
    |> List.ofSeq

## Run experiments

Seed the process with an empty list. The output will be a `Distribution`.

In [63]:
enumerate [] montyHallStay

seq [(Car, 0.3333333333); (Goat, 0.3333333333); (Goat, 0.3333333333)]

============ ProbabilisticComputation is a Sample
PC is a Return 1: Car(0.333333)
PC is a Return 1: Goat(0.333333)
PC is a Return 1: Goat(0.333333)
============ ProbabilisticComputation is a Sample
PC is a Return 1: Car(0.333333)
PC is a Return 1: Goat(0.333333)
PC is a Return 1: Goat(0.333333)
============ ProbabilisticComputation is a Sample
PC is a Return 1: Car(0.333333)
PC is a Return 1: Goat(0.333333)
PC is a Return 1: Goat(0.333333)


In [59]:
let resultStay = enumerate [] montyHallStay |> summarize

============ comp is a Sample
1: Car(0.333333)
1: Goat(0.333333)
1: Goat(0.333333)


In [57]:
resultStay

[(Car, 0.3333333333); (Goat, 0.6666666667)]

In [58]:
let resultSwitch = enumerate [] montyHallSwitch |> summarize

============ comp is a Sample
============ comp is a Sample
1: Car(0.333333)
2: Goat(1.000000)
============ comp is a Sample
1: Goat(0.333333)
2: Car(1.000000)
============ comp is a Sample
1: Goat(0.333333)
2: Car(1.000000)


In [19]:
resultSwitch

[(Goat, 0.3333333333); (Car, 0.6666666667)]

## Further reading

Gabasova, Evelina. How to Look like a Statistician: A Developer’s Guide to Probabilistic Programming, 2018. https://channel9.msdn.com/Events/FSharp-Events/fsharpConf-2018/06.   
———. Probabilistic-Programming: [Talk] How to Look like a Statistician: A Developer’s Guide to Probabilistic Programming. F#, 2018. https://github.com/evelinag/probabilistic-programming.

Syme, Don, Adam Granicz, and Antonio Cisternino. Expert F# 4.0. Apress, 2015.

Forkmann, Steffen. “Some special monads in F# – Part 4 of n – Application: The Monty Hall problem » Rash thoughts about .NET, C#, F# and Dynamics NAV.,” 2011. http://www.navision-blog.de/blog/2011/08/16/some-special-monads-in-f-part-4-of-n-application-the-monty-hall-problem/.   
Kidney, Donnacha Oisín. Monty-Hall . Swift, 2015. https://github.com/oisdk/Monty-Hall.   
Microsoft Research. Infer.Net Framework - Monty Hall. C#, 2016. https://github.com/ledohod/Infer.Net.   
Purcell. Monty Hall Problem Simulated in Haskell. https://gist.github.com/purcell/5957243.  
Silva, André. Pfp-Scala: Probabilistic Functional Programming in Scala. Shell, 2014. https://github.com/andresilva/pfp-scala.   


Erwig, Martin, and Steve Kollmansberger. “Functional Pearls: Probabilistic Functional Programming in Haskell.” Journal of Functional Programming 16, no. 01 (2006): 21–34. http://web.engr.oregonstate.edu/~erwig/papers/PFP_JFP06.pdf